# Ask a youtube video

### Install dependencies

In [90]:
#!pip install langchain
#!pip install langchain-community
#!pip install python-dotenv
#!pip install langchain-openai
#!pip install weaviate-client
#!pip install openai
#!pip install youtube_transcript_api

### Load .env variables

In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

YOUR_OPENAI_KEY = os.getenv("YOUR_OPENAI_KEY")
YOUR_WEAVIATE_KEY = os.getenv("YOUR_WEAVIATE_KEY")
YOUR_WEAVIATE_URL = os.getenv("YOUR_WEAVIATE_URL")

# 1. Fetch the video information

In [3]:
from youtube_transcript_api import YouTubeTranscriptApi

"""
This function extracts the video id from the youtube link
:param youtube_link: the youtube link
:return: the video id
"""
def get_video_id(youtube_link: str) -> str:
    video_id = youtube_link.split("v=")
    if len(video_id) > 1:
        return video_id[1]
    video_id = youtube_link.split("youtu.be/")
    if len(video_id) > 1:
        return video_id[1]
    
    return youtube_link.split("/")[-1]

"""
This function fetches the transcript of a youtube video
:param youtube_link: the youtube link
:return: the transcript of the video
"""
def fetch_transcript(youtube_link : str) -> str:

    video_id = get_video_id(youtube_link)

    # I could have added even more languages
    transcript = YouTubeTranscriptApi.get_transcript(video_id, languages=['en','fr'])

    return " ".join([line['text'] for line in transcript])

In [9]:
video_urls = [  "https://youtube.com/shorts/cFp4bQTtb1M?si=y_yC79dUkNveBndR",
              "https://youtube.com/shorts/3JOTPH3izlA?si=KXNBMNZ-wz6A7Ktd",
               "https://youtube.com/shorts/OLCLfJJXFPM?si=8tVYvvafQ-_77ZOK" ] #"https://www.youtube.com/live/RXeOiIDNNek?si=DjyZK7jnuCYbbXg3" ] # WWDC 2024 Keynote]

video_transcripts = [ fetch_transcript(video_url) for video_url in video_urls ]

In [10]:
video_transcripts

["quelles sont les marques de voitures qui ont le plus d'accidents en France en moyenne un accident survient tous les 160000 km dans l'Hexagone et la société car vertical spécialisée dans le traitement des données et des historiques automobiles a présenté les résultats de son étude sur les modèles de véhicules les plus accidentés dans l'Hexagone l'an dernier alors perso avant de lire le classement j'avais déjà ma petite idée de quelle marque avaient les conducteurs les plus débiles mais je te laisse juger tout en bas du classement Mazda Fiat et Renault sont des marques très peu impliquées dans des accidents en 13e position se trouve alpha Romeo juste après Toyota eux à la 12e place le géant Mercedes et lui 11e je reconnais ça m'a un peu surpris enfin le top 10 est le suivant Porsche est 10e seat 9e Peugeot 8e Opel 7e Volkswagen 6e Land Rover et Citroën sont 5 et 4e quant au top 3 donc les marques qui se retrouvent dans le plus d'accidents en France il est composé d'Audi qui est vraimen

# 2. Split transcript into sentences

In [11]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Split the text into chunks of 800 characters with an overlap of 100 characters
text_splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=100)
split_transcripts = [ text_splitter.split_text(transcript) for transcript in video_transcripts ]

# 3. Vector DB Storage

In [12]:
import weaviate
import weaviate.classes as wvc
from langchain.vectorstores import Weaviate

# Connect to Weaviate
client = weaviate.connect_to_weaviate_cloud(
    cluster_url=YOUR_WEAVIATE_URL,
    auth_credentials=weaviate.auth.AuthApiKey(api_key=YOUR_WEAVIATE_KEY),
    headers = {
        "X-OpenAI-Api-Key": YOUR_OPENAI_KEY
    }
)

print("Client is ready: ", client.is_ready())

# Deletes previously created collections
client.collections.delete_all()

try:
    # Create the Transcript collection
    collection = client.collections.create(
        name="Transcript",
        vectorizer_config=wvc.config.Configure.Vectorizer.text2vec_openai(
            model="text-embedding-3-small"  # Specify the model
        ),
        properties=[
            wvc.config.Property(
                name="text",
                data_type=wvc.config.DataType.TEXT,  # Data type for the property
                description="Text of the transcript"  # Description of the property
            ),
        ]
    )
finally:
    client.close()

# Deprecated way to store using vectorstore
client = weaviate.Client(url=YOUR_WEAVIATE_URL, auth_client_secret=weaviate.auth.AuthApiKey(YOUR_WEAVIATE_KEY), additional_headers={"X-OpenAI-Api-Key": YOUR_OPENAI_KEY})
vectorstore = Weaviate(client, "Transcript", "text", attributes=["source"])

Client is ready:  True


/home/dup/Documents/Pro/InsideBoardAI/AskYourVideo/.venv/lib/python3.12/site-packages/weaviate/warnings.py:162: DeprecationWarning: Dep016: Python client v3 `weaviate.Client(...)` connections and methods are deprecated. Update
            your code to use Python client v4 `weaviate.WeaviateClient` connections and methods.

            For Python Client v4 usage, see: https://weaviate.io/developers/weaviate/client-libraries/python
            For code migration, see: https://weaviate.io/developers/weaviate/client-libraries/python/v3_v4_migration
            
  warnings.warn(


In [13]:
# Load text into the vector store

text_meta_pair = []
for transcript, video_url in zip(split_transcripts, video_urls):
    for chunk in transcript:
        text_meta_pair.append((chunk, {"source": video_url}))
texts, metas = zip(*text_meta_pair)
vectorstore.add_texts(texts, metas)

['8ad6b28c-00ae-401f-b5a2-3976ca58e26c',
 '3c527bc5-91d5-4d1c-b141-20d656cdae05',
 '72f37811-e77e-40d2-861b-2eac6ad77580',
 '28a76988-122a-41f4-a272-2b56edde5c43',
 '68891ac2-3fe9-4b92-bdd9-84ca9f2c34ae',
 '6788549e-2c47-4910-b2b4-30224fa61567']

# 4. Similarity Search

In [19]:
# query = "What's new on ios18 ? "
# query = "Réponds en Français. Tell me about Apple Intelligence."
# query = "Have you heard about a way to create custom emojis ?"
#query = "Do you think that all that AI in our everyday life devices could be harmful ?"
query = "Peux-tu me donner le top 3 des voitures les plus accidentées en France ?"


# retrieve chunks related to the query
vect = vectorstore.similarity_search(query, k=3)
vect

[Document(page_content="quelles sont les marques de voitures qui ont le plus d'accidents en France en moyenne un accident survient tous les 160000 km dans l'Hexagone et la société car vertical spécialisée dans le traitement des données et des historiques automobiles a présenté les résultats de son étude sur les modèles de véhicules les plus accidentés dans l'Hexagone l'an dernier alors perso avant de lire le classement j'avais déjà ma petite idée de quelle marque avaient les conducteurs les plus débiles mais je te laisse juger tout en bas du classement Mazda Fiat et Renault sont des marques très peu impliquées dans des accidents en 13e position se trouve alpha Romeo juste après Toyota eux à la 12e place le géant Mercedes et lui 11e je reconnais ça m'a un peu surpris enfin le top 10 est le suivant Porsche est 10e", metadata={'source': 'https://youtube.com/shorts/cFp4bQTtb1M?si=y_yC79dUkNveBndR'}),
 Document(page_content="et lui 11e je reconnais ça m'a un peu surpris enfin le top 10 est 

# 5. My Custom Chatbot

In [20]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI

# define chain
chain = load_qa_chain(
    OpenAI(openai_api_key = YOUR_OPENAI_KEY, temperature=0.1),
    chain_type="stuff"
)

/home/dup/Documents/Pro/InsideBoardAI/AskYourVideo/.venv/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [21]:
# create answer
answer = chain.run(input_documents=vect, question=query)
answer

/home/dup/Documents/Pro/InsideBoardAI/AskYourVideo/.venv/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
/home/dup/Documents/Pro/InsideBoardAI/AskYourVideo/.venv/lib/python3.12/site-packages/pydantic/main.py:1070: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.7/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
